# Project







## Link to github repository
https://github.com/X9126/dsci-100-2025ss-individual/tree/main

## Introduction
The video game industry has seen major growth, with players of all ages joining in. To keep players engaged, developers and marketers are paying closer attention to player behavior, especially when it comes to game newsletters. Understanding which types of players are most likely to subscribe can help tailor communication strategies and boost long-term player retention.



## Question 
Can experience level, age, gender, and total play time be used to predict newsletter subscription in the players dataset?


## Data Description
This analysis uses the `players.csv` dataset, which contains `196` records and `7` variables describing individual game players. The dataset was provided by a research group at `UBC` studying player behavior in a `Minecraft` server environment. The dataset includes both `categorical` and `numerical` variables. These variables are: `experience`, a `categorical` variable indicating the player’s self-reported gaming experience (e.g., `Pro`, `Veteran`); `subscribe`, a `boolean` variable showing whether the player subscribed to a game-related newsletter; `played_hours`, a `numerical` variable representing the total hours played; `Age`, a `numerical` variable recording the player’s age; `gender`, a `categorical` variable for gender identity; and two identifier variables, `name` and `hashedEmail`, which are excluded from analysis.

Among the `numerical` variables, `played_hours` ranges from `0` to `95` hours, with a mean of about `9.8` and a median of around `2.0`, indicating a right-skewed distribution. The `Age` variable ranges from `8` to `29`, with an average of about `17.3` and `2` missing values that need to be handled before modeling. The `categorical` variables such as `experience` and `gender` will require encoding during analysis. This project will use `experience`, `played_hours`, `gender`, and `Age` as explanatory variables to predict the binary target variable `subscribe`. The dataset is suitable for both `prediction` and `player-type comparison`, as `experience` can also serve as a grouping variable. Overall, the data is clean, organized, and mostly complete, with only minimal preprocessing needed before modeling.

## Methods & Results

In [2]:
library(tidyverse)    
library(tidymodels) 

── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Use suppressPackageStartupMessages() to eliminate package startup messages



In [3]:
player <- read_csv("players.csv") 
head(player) 

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17


In [4]:
clean_player <- player |> 
  select(-name, -hashedEmail)

head(clean_player)

experience,subscribe,played_hours,gender,Age
<chr>,<lgl>,<dbl>,<chr>,<dbl>
Pro,TRUE,30.3,Male,9
Veteran,TRUE,3.8,Male,17
Veteran,FALSE,0.0,Male,17
Amateur,TRUE,0.7,Female,21
Regular,TRUE,0.1,Male,21
Amateur,TRUE,0.0,Female,17


In [5]:
clean_player |> 
  group_by(gender) |> 
  summarize(count = n())

gender,count
<chr>,<int>
Agender,2
Female,37
Male,124
Non-binary,15
Other,1
Prefer not to say,11
Two-Spirited,6


In [8]:
clean_player |> 
  group_by(experience) |> 
  summarize(count = n())

experience,count
<chr>,<int>
Amateur,63
Beginner,35
Pro,14
Regular,36
Veteran,48
